In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from jupyterthemes import jtplot
from sklearn.metrics import auc, roc_curve, roc_auc_score
%matplotlib inline
jtplot.style()
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

In [4]:
train_data = pd.read_csv('train.csv', index_col='Id')
test_data = pd.read_csv('test.csv', index_col='Id')
y_test = pd.read_csv('sample_submission.csv', index_col='Id')

In [5]:
train_data.fillna(-999, inplace=True)
test_data.fillna(-999, inplace=True)

In [6]:
#Находим категориальные признаки
cat_feat = list(train_data.dtypes[train_data.dtypes == object].index)

#отфильтруем непрерывные признаки
num_feat = [f for f in train_data if f not in (cat_feat + ['ID', 'SalePrice'])]

# Смотрим сколько у нас значений по каждому категориальному признаку
cat_nunique = train_data[cat_feat].nunique()

#Чтобы в разы не увеличивать число признаков при построении dummy,
#будем использовать категориальные признаки с < 30 уникальных значений
cat_feat = list(cat_nunique[cat_nunique < 30].index)

In [7]:
# Создаем дамми-переменные для категорий
dummy_train = pd.get_dummies(train_data[cat_feat], columns=cat_feat)
dummy_test = pd.get_dummies(test_data[cat_feat], columns=cat_feat)

dummy_cols = list(set(dummy_train) & set(dummy_test))

dummy_train = dummy_train[dummy_cols]
dummy_test = dummy_test[dummy_cols]

In [8]:
# Заменяем пропуски на специальное значение -999, чтобы деревья могли их отличить
X_train = pd.concat([train_data[num_feat],
                     dummy_train], axis=1)

X_test = pd.concat([test_data[num_feat],
                     dummy_test], axis=1)

In [9]:
y_train = train_data['SalePrice'] #создаем целевую переменную

In [10]:
lr_rf = RandomForestRegressor(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1) #создаем объект случайного леса регрессора
lr_rf.fit(X_train, y_train) 

RandomForestRegressor(max_depth=5, max_features=0.5, min_samples_leaf=20,
                      n_estimators=10, n_jobs=-1)

In [54]:
y_pred_train_rf = lr_rf.predict(X_train) #делаем предсказание на тренировочной выборке
y_pred_test_rf = lr_rf.predict(X_test) #делаем предсказание на тестоврй выборке

In [55]:
lr_rf.score(X_train, y_train)

0.8372584587771146

In [56]:
lr_rf.score(X_test, y_test['SalePrice'])

-14.486579595278323

In [57]:
#Определим важность признаков
imp = pd.Series(lr_rf.feature_importances_)
imp.sort_values(ascending=False)

3      0.393996
238    0.136599
25     0.135886
11     0.082849
15     0.065396
         ...   
179    0.000000
178    0.000000
177    0.000000
176    0.000000
142    0.000000
Length: 285, dtype: float64

Наиболее важным признаком для модели является 3-ий, площадь лота

Кросс-валидация

In [61]:
cv_score_train = cross_validate( lr_rf, X_train, y_train, cv = 10, n_jobs=-1 ) #проверяем на тренировочной выборке
cv_score_train

{'fit_time': array([0.06283045, 0.06682014, 0.04388118, 0.04687357, 0.06482577,
        0.04388118, 0.04487896, 0.04388118, 0.04487896, 0.0658226 ]),
 'score_time': array([0.00797915, 0.00698113, 0.0209446 , 0.01994658, 0.00698066,
        0.0209446 , 0.02094364, 0.0209446 , 0.02094364, 0.00598383]),
 'test_score': array([0.80604295, 0.82148529, 0.88451011, 0.80198736, 0.75806992,
        0.79304311, 0.83553892, 0.81670369, 0.72019615, 0.83208153])}

In [62]:
cv_score_train['test_score'][cv_score_train['test_score'].argmax()] #Выводим наилучший результат для тренировочной выборки.

0.884510114567754

In [63]:
cv_score_test = cross_validate( lr_rf, X_test, y_test['SalePrice'], cv = 10, n_jobs=-1 ) #проверяем на тестовой выборке
cv_score_test

{'fit_time': array([0.02991962, 0.02892256, 0.03789854, 0.03889585, 0.03989339,
        0.03989339, 0.03590441, 0.04388285, 0.04687452, 0.04987311]),
 'score_time': array([0.0179522 , 0.0179522 , 0.00797868, 0.00698137, 0.00698113,
        0.00797844, 0.0129652 , 0.0119741 , 0.01196766, 0.01196194]),
 'test_score': array([0.8091047 , 0.88853379, 0.83497817, 0.89116156, 0.80570202,
        0.69744484, 0.89408473, 0.74560569, 0.90733064, 0.74342982])}

In [64]:
cv_score_test['test_score'][cv_score_test['test_score'].argmax()] #Выводим наилучший результат для тестовой выборки.

0.9073306435548206

Стэккинг

In [65]:
train_median = train_data[num_feat].median()

X_train = pd.concat([train_data[num_feat].fillna(train_median),
                     train_data[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_train], axis=1)

X_test = pd.concat([test_data[num_feat].fillna(train_median),
                     test_data[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_test], axis=1)

scaler = StandardScaler()
scaler.fit(X_train[num_feat])

X_train[num_feat] = scaler.transform(X_train[num_feat])
X_test[num_feat] = scaler.transform(X_test[num_feat])

In [66]:
regressor = StackingRegressor(
    [
        ('li', LinearRegression()),
        ('dt', DecisionTreeRegressor()),
        ('rf', RandomForestRegressor())
    ],
LinearRegression())

In [67]:
regressor.fit(X_train, y_train)

StackingRegressor(estimators=[('li', LinearRegression()),
                              ('dt', DecisionTreeRegressor()),
                              ('rf', RandomForestRegressor())],
                  final_estimator=LinearRegression())

In [69]:
y_pred_st_train = regressor.predict(X_train)
y_pred_st_train

array([206778.62145849, 176923.02211008, 223858.7034647 , ...,
       261578.14603047, 139841.3477503 , 148037.42913415])

In [70]:
y_pred_st_test = regressor.predict(X_test)
y_pred_st_test

array([128306.64560425, 153920.02228326, 181973.6549212 , ...,
       150208.27374425, 106752.23613271, 228469.82488431])

In [71]:
cv_st_score_train = cross_validate( regressor, X_train, y_train, cv = 10, n_jobs=-1 ) #проверяем на тренировочной выборке
cv_st_score_train

{'fit_time': array([15.40383363, 15.27617502, 15.31207895, 14.53615355, 15.44176364,
        14.93109727, 14.92710805, 15.41380548, 14.94306493, 15.07072282]),
 'score_time': array([0.01396251, 0.01495934, 0.01495934, 0.01595712, 0.01692295,
        0.02593064, 0.01695442, 0.01399779, 0.01695514, 0.01495934]),
 'test_score': array([0.86174922, 0.8876374 , 0.92793107, 0.75160396, 0.89039321,
        0.89008513, 0.88568807, 0.88628669, 0.76849213, 0.85830779])}

In [72]:
cv_st_score_train['test_score'][cv_st_score_train['test_score'].argmax()] #Выводим наилучший результат для тренировочной выборки.

0.9279310728318642

In [73]:
cv_st_score_test = cross_validate( regressor, X_test, y_test['SalePrice'], cv = 10, n_jobs=-1 ) #проверяем на тестовой выборке
cv_st_score_test

{'fit_time': array([15.8955512 , 15.98531079, 15.95938301, 16.08405566, 15.86463428,
        16.20173335, 16.12390804, 16.07703304, 16.52683163, 16.13488221]),
 'score_time': array([0.01397014, 0.01396465, 0.01495767, 0.01295924, 0.01498032,
        0.01298666, 0.01599097, 0.01496029, 0.01495981, 0.01592231]),
 'test_score': array([1.        , 1.        , 1.        , 1.        , 0.99999997,
        1.        , 1.        , 1.        , 1.        , 1.        ])}

In [74]:
cv_st_score_test['test_score'][cv_st_score_test['test_score'].argmax()] #Выводим наилучший результат для тестовой выборки.

1.0

При кросс-валидации с использованием ансамблей моделей точность на тестовой выборке достигает 100%, что превышает точность без ансамблей.